In [121]:
import geopandas as gpd
import turfpy
import folium
import numpy as np
import shapely
import geemap
import ipywidgets as widgetsW
from shapely.geometry import Point as shppt
from geojson import Point, Feature, FeatureCollection
from ipyleaflet import Map, GeoJSON, LayersControl, Marker, LayerGroup
from ipywidgets import Label
from ipyleaflet import WidgetControl
from turfpy.measurement import nearest_point, distance
import ee

In [67]:
map = geemap.Map()
ee.Initialize()

In [76]:
geojson_path = "E:/Satyukt/paired_pilot/data/shp/VILLAGE.shp"

gdf = gpd.read_file(geojson_path)
gdf = gdf.loc[:100]

## Calculate the Centroid
gdf['centroid'] = [x.centroid for x in gdf.geometry]

#For meters
# gdf = gdf.to_crs(3763)

In [147]:
## Initialize the Map
m = Map(center = [18.42,77.35], zoom = 12)

# Convert the centroids in gdf to a FeatureCollection
feature_collection = FeatureCollection([Feature(geometry=Point(list(gdf['centroid'][i].coords)[0])) for i in range(len(gdf))])

In [72]:

def getNearestPointDistance(pt,dist):
    '''
    Returns the Nearest Point in the Feature collection
    It takes two arguments:
        pt >> shapely.coordinates.sequence
        dist >> Its the boolean value which if set to true, 
        Returns the distance in meters to that point
    '''
    global feature_collection
    point = Point(pt)
    if dist == True:
        return (distance(point,nearest_point(point, feature_collection)))
    
    return nearest_point(point, feature_collection)




def bufferPoint(pt):
    '''
    Takes the Shapely.geometry.point object and returns the buffer of 50 meters
    '''
    return gpd.GeoSeries(pt).buffer(0.2)




def pointsInBuffer(buff) -> shapely.geometry.polygon:
    '''
    Takes the Buffer of a point and returns all the points which lies inside of the Polygon
    It returns the GeoDataFrame of the Points.
    '''
    global gdf
    return gdf.iloc[[buffer.contains(gdf['centroid'][i])[0] for i in range(len(gdf))]] 




def convShapelyGeometry(coords) -> shapely.geometry.point.Point:
    '''
    Convert the Shapely.geometry.Point object to ee.Geometry.Point object
    '''
    return ee.Geometry.Point(list(coords.coords)[0])




def getNDVI(coords):
    '''
    Calculate the NDVI value of the first image for the ee.Geometry.Point object
    Returns NDVI value
    '''
    coords = convShapelyGeometry(coords)
    
    s2_collection = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(coords).first()
    
    ndvi = s2_collection.normalizedDifference(['B8', 'B4'])
    
    # Get the value of NDVI at the point of interest
    point_ndvi = ndvi.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=coords,
        scale=10
    )

    ndvi_value = point_ndvi.get('nd').getInfo()
    return ndvi_value



def markerCoords(cords):
    '''
    This function takes the coordinates of the Centroid.
    Returns the Marker object
    '''
    return Marker(location = (cords.coords[0][1],cords.coords[0][0])) 
    

In [133]:
gdf['Markers'] = gdf['centroid'].apply(markerCoords)
add_markers = [m.add_layer(mark) for mark in gdf.loc[:50]['Markers']]

clicked_lat = None
clicked_lon = None
def handle_interaction(**kwargs):
    global m
    lat,lon = kwargs.get('coordinates')
    clicked_lat = lat
    clicked_lon = lon
    if kwargs.get('type') == 'click':
        # Map.default_style = {'cursor': 'wait'}
        # with output_widget:
        #     output_widget.clear_output()
        #     print(lat,lon)
        # Map.default_style = {'cursor': 'pointer'}
        points_in_buff = pointsInBuffer(bufferPoint(Point(lat,lon)))
        for marker in points_in_buff['Markers']:
            marker.opacity = 1.0   

In [134]:
m.on_interaction(handle_interaction)
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
m.add_control(output_control)

In [149]:
### Add all the points in the dataframe on Map
### On clicking the a point get all the points on that buffer
### Returned Geodataframe - Highlight the centroid under buffer

In [154]:
def handle_click(**kwargs):
    print(kwargs)
    lat,lon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        print(lat, lon)
        # m.add_layer(Marker(location=kwargs.get('coordinates')))

m.on_interaction(handle_click)

In [165]:
from ipyleaflet import Map, Marker, Popup

def handle_click(event, **kwargs):
    lat, lon = event['coordinates']
    popup = Popup(location=(lat, lon), child=HTML('You clicked at ({:.4f}, {:.4f})'.format(lat, lon)))
    m.add_layer(popup)

latitude, longitude = 40.7128, -74.0060  # New York City coordinates
zoom_level = 10

m = Map(center=(latitude, longitude), zoom=zoom_level)

marker = Marker(location=(latitude, longitude))
marker.on_click(handle_click)
m.add_layer(marker)

m


Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [164]:
gdf.groupby('SUB_DIST11')['ndvi'].mean()
gdf

,KARNA_ID,NAME_11,SUB_DIST11,DISTRICT11,STATE_UT,CENSUS2011,geometry,centroid,ndvi,Markers
0,1,Mukhed,Aurad,Bidar,Karnataka,2955805456599970,"POLYGON ((77.26762 18.45161, 77.27471 18.47334...",POINT (77.28518620189236 18.461010115602893),-0.042930,"Marker(location=[18.461010115602893, 77.285186..."
1,2,Bawalgaon,Aurad,Bidar,Karnataka,2955805456599972,"POLYGON ((77.33392 18.42091, 77.33502 18.42085...",POINT (77.3589626888698 18.42177201032195),-0.047967,"Marker(location=[18.42177201032195, 77.3589626..."
2,3,Nandi Bijalgaon,Aurad,Bidar,Karnataka,2955805456599971,"POLYGON ((77.27727 18.42520, 77.27937 18.42662...",POINT (77.30227896672584 18.42363933857228),-0.050864,"Marker(location=[18.42363933857228, 77.3022789..."
3,4,Chikli Udgir,Aurad,Bidar,Karnataka,2955805456599969,"POLYGON ((77.27727 18.42520, 77.27745 18.42483...",POINT (77.26165389523375 18.40081223598321),-0.042760,"Marker(location=[18.40081223598321, 77.2616538..."
4,5,Hokrana,Aurad,Bidar,Karnataka,2955805456599973,"POLYGON ((77.32410 18.42190, 77.32428 18.42189...",POINT (77.33008207087087 18.400810467095308),-0.053225,"Marker(location=[18.400810467095308, 77.330082..."
...,...,...,...,...,...,...,...,...,...,...
96,94,Hippalgaon,Aurad,Bidar,Karnataka,2955805456600030,"POLYGON ((77.32156 18.18112, 77.32357 18.18138...",POINT (77.34519872977849 18.177207820983977),-0.024486,"Marker(location=[18.177207820983977, 77.345198..."
97,95,Lingadhalli(Karamungi),Aurad,Bidar,Karnataka,2955805456600085,"POLYGON ((77.46760 18.19992, 77.46793 18.20017...",POINT (77.48012493661227 18.191809508673472),-0.056233,"Marker(location=[18.191809508673472, 77.480124..."
98,96,Maskal,Aurad,Bidar,Karnataka,2955805456600032,"POLYGON ((77.39236 18.19836, 77.39350 18.19907...",POINT (77.40261643156724 18.175010528295303),-0.050874,"Marker(location=[18.175010528295303, 77.402616..."
99,97,Bedkunda,Aurad,Bidar,Karnataka,2955805456600023,"POLYGON ((77.30103 18.20014, 77.30176 18.20026...",POINT (77.3052530793358 18.17994094361245),-0.043763,"Marker(location=[18.17994094361245, 77.3052530..."
